# Install Java

In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y

In [ ]:
!java -version

In [ ]:
!readlink -f $(which java)

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install HBase

In [ ]:
!wget https://downloads.apache.org/hbase/2.5.10/hbase-2.5.10-bin.tar.gz
!tar xvf hbase-2.5.10-bin.tar.gz

In [ ]:
! mv hbase-2.5.10 /opt/hbase

In [4]:
!sed -i 's|# export JAVA_HOME=.*|export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64|' /opt/hbase/conf/hbase-env.sh

In [ ]:
# Verify JAVA_HOME
import os

# Check if JAVA_HOME is set, if not set it to the correct path
java_home = '/usr/lib/jvm/java-11-openjdk-amd64'
if not os.path.exists(java_home):
    raise FileNotFoundError(f'Java path not found: {java_home}')
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = java_home + '/bin:' + os.environ['PATH']


!java -version

In [ ]:
#Start HBase
!/opt/hbase/bin/start-hbase.sh

In [ ]:
!jps

# Install Spark

In [ ]:
!wget -v https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [ ]:
!tar xvf spark-3.4.3-bin-hadoop3.tgz

In [ ]:
!pip3 install findspark --break-system-packages

In [ ]:
! mv spark-3.4.3-bin-hadoop3 /opt/spark

In [3]:
import os
import findspark

os.environ["SPARK_HOME"] = "/opt/spark"

# Initialize PySpark
findspark.init()


# Collect data

Download csv file from https://obis.org/data/access/#

In [ ]:
!echo "create 'OBIS', 'cf'" | /opt/hbase/bin/hbase shell

In [6]:
from pyspark.sql import SparkSession

# Start Spark Session with HBase connector
spark = SparkSession.builder \
    .appName("HBase-PySpark-Connection") \
    .config("spark.jars.packages", "org.apache.hbase.connectors.spark:hbase-spark:1.0.1") \
    .getOrCreate()

In [ ]:
df = spark.read \
    .format("org.apache.hbase.connectors.spark") \
    .option("hbase.table.name", "OBIS") \
    .load()

df.show()

In [12]:
spark.stop()

In [ ]:
#Stop HBase
!/opt/hbase/bin/stop-hbase.sh

In [ ]:
!jps